In [122]:
import cirq
import numpy as np
import scipy as sp

After seeing our first example of how one can use a quantum circuit to solve the Deutsch(-Jozsa) Problem in a more efficient way (but only slightly more efficient than a classical probabilistic computer), we now turn to a problem where the runtime on a classical computer is exponential with $\mathcal{O}(2^n/2)$, while using a quantum circuit together with classical post processing the problem can be solved in $\mathcal{O}(n)$

Reading Ch.6, SIAM J. Comput., 26(5), 1474–1483

\begin{problem}
Given a function $f: \{0,1\}^n \rightarrow \{0,1\}^n$, such that there exists $\mathbf{s} = s_1s_2...s_n$ so that $f(\mathbf{x}) = f(\mathbf{y}) \iff (\mathbf{x} = \mathbf{y}\, \mathrm{or} \, \mathbf{x}= \mathbf{y} \oplus \mathbf{s})$. 

Determine the $\mathbf{s}$ 
\end{problem}
Note, that we can see $\{0,1\}^n$ as the a vector space over $Z_2$ this vector space has dimension $n$ (one basis is $\{(\delta_{i,k})_{i\in (0,1,..,n) )}\}$. This is not the same as the (complex) Hilbert space of the qubits $(\mathbb{C}^2)^{\otimes n}$, which has dimension $2^n$.

Before we write down Simon's algorithm, we discuss some identities that will help with understanding the algorithm. 

We start by applying $H^{\otimes n}$ to $|0\rangle^{\otimes n}$ 
\begin{equation}
H^{\otimes n}|0\rangle^{\otimes n} = \frac{1}{2^{n/2}}\left(|0\rangle+|1\rangle \right)^{\otimes n}=\frac{1}{2^{n/2}} \sum_{x \in \{0,1\}}|x\rangle\left(|0\rangle+|1\rangle \right)^{\otimes (n-1)} = ... =\frac{1}{2^{n/2}}\sum_{\mathbf{x} \in \{0,1\}^n}|\mathbf{x}\rangle 
\end{equation}
Let ${\bf x,y} \in A_n = \{0,1\}^n $ and $\mathbf{s} = \mathbf{x}\oplus \mathbf{y}$ (by modular arithmetic this implies $\mathbf{x}\oplus\mathbf{s} =  \mathbf{y}$). We now apply $H^{\oplus n}$ to the superposition of $\bf x, y$
First we note that if $x\in \{0,1\}^n$
\begin{equation}
H^{\otimes n}|\mathbf{x}\rangle = H|x_1\rangle H|x_2\rangle... H|x_n\rangle = \frac{1}{2^{n/2}}\left(|0\rangle +(-1)^{x_1}|1\rangle\right)\left(|0\rangle +(-1)^{x_2}|1\rangle \right)...\left(|0\rangle +(-1)^{x_n}|1\rangle\right) = \frac{1}{2^{n/2}}\sum_{z\in A_n}\left(-1\right)^{\bf x \cdot z} |\mathbf{z}\rangle
\end{equation}
using this result we immediately find 
\begin{equation}
H^{\oplus n}\frac{1}{\sqrt{2}}\left(|\mathbf{x}\rangle +|\mathbf{y}\rangle \right) = \frac{1}{2^{(n+1)/2}}\sum_{z\in A_n}(\left(-1\right)^{\bf x \cdot z}+\left(-1\right)^{(\mathbf{x}\oplus\mathbf{s})\cdot \mathbf{z}} )|\mathbf{z}\rangle
\end{equation}
By inspection it becomes clear that 
\begin{equation}
(\left(-1\right)^{\bf x \cdot z}+\left(-1\right)^{(\mathbf{x}\oplus\mathbf{s})\cdot \mathbf{z}}) = \left(-1\right)^{\bf x \cdot z} \left(1+\left(-1\right)^{\bf s\cdot z }\right) = \begin{cases}
1 &\text{$z \in \{s\}^\perp$}\\
0 &\text{else}
\end{cases}
\end{equation}
Putting it all together gives
\begin{equation}
H^{\oplus n}\frac{1}{\sqrt{2}}\left(|\mathbf{x}\rangle +|\mathbf{y}\rangle \right) = \frac{1}{2^{(n-1)/2}}\sum_{z \in\{s\}^\perp } \left(-1\right)^{\bf x \cdot z}|\mathbf{z}\rangle
\end{equation}
This effectively means that after applying $H^{\otimes n}$ to a state of the from $|{\bf x }\rangle+|{\bf x\oplus s}\rangle$ the system is in a uniform superposition of all states belonging to $\{\mathbf{s}\}^\perp$. One can now devise a strategy to solve Simon's problem. Namely, we use a quantum circuit to give us a state that is a uniform superposition of states in $\{\bf s\}^\perp$ and then use this circuit to sample $n-1$ linearly independent vectors spanning $\{\bf s\}^\perp$. From linear algebra, we know that by finding the orthogonal complement to $\{\mathbf{s}\}^\perp$ (which can be easily done by solving a linear system), we find $\bf s$. The exact algorithm is summarised below.

\begin{theorem}
Simon's problem can be solved with expected run-time in the quantum operation $\mathcal{O}(n)$ plus $\mathcal{O}(n^3)$ additional classical operation (in total polynomial run time) by applying the following algorithm.

1. i =0
2. Prepare $|\psi_1\rangle = \frac{1}{\sqrt{2}^n} \sum_{x\in A_n} |\mathbf{x}\rangle|\mathbf{0}\rangle$ 
($|\psi_1\rangle \in Z^{\otimes 2n}_2$)
3. Apply $cU_f$,  $|\psi_2\rangle =cU_f |\psi_1\rangle =\frac{1}{\sqrt{2}^n} \sum_{x\in A_n} |\mathbf{x}\rangle|f(\mathbf{x})\rangle$ 
4. Apply $H^{\oplus n}$ to the first register (first $n$ qubits)
5. Measure the first register and record $\mathbf{w}_i \in Z^{\otimes n}_2$
6. if: $\mathrm{dim}\left(\mathrm{span}(w_j)_{j\in\{0,1,...,i\}}\right) = n-1$ go to 7, else: $i \mathrel{+}=1$ go to 2.
7. (Classical) Solve $W \mathbf{s}^\top = 0 \mod 2$, 
8. Output unique non-zero solution
\end{theorem}

To see why this algorithm works, we only have to combine the results from above. 
As usual we start with the state $|0 \rangle^{\otimes 2n}$ and then apply $H^\otimes n$ to obtain $|\psi_1\rangle$. The application of $cU_f$ now works in the same way as already seen for the Deutsch problem. 
We are now going to use that $\{0,1\}^n$ can be partitioned into $2^{n-1}$ pairs of the form $\{\mathbf{x},\mathbf{x}\oplus \mathbf{s}\}$. One can then rewrite $|\psi_2\rangle$ in the following form
\begin{equation}
|\psi_2\rangle =\frac{1}{{2}^{(n-1)/2}} \sum_{x\in I} \left(|\mathbf{x}\rangle|f(\mathbf{x})\rangle + |\mathbf{x} \oplus \mathbf{s}\rangle|f(\mathbf{x}\oplus \mathbf{s})\rangle\right) = \frac{1}{2^{(n-1)/2}} \sum_{x\in I} \left(|\mathbf{x}\rangle + |\mathbf{x} \oplus \mathbf{s}\rangle\right) |f(\mathbf{x})\rangle
\end{equation}
where we have used that $f(\mathbf{x}\oplus \mathbf{s}) =f(\mathbf{x}) $ and denoted the subset consisiting of one representative of each pair by $I$ (note which one we chose does not matter since if we chose $\bf x \oplus s$ have $x = \bf x \oplus s \oplus s$). 
From (5), we know that after applying the second set of $H$-gates we will obtain a uniform superposition in $\{\bf s\}^\perp$ and measuring those, we will get an element $\mathbf{w}_i \in \{\bf s\}^\perp$.

Comment 1: If it is not entirely clear how the superposition after 4 will look like, imagine that we had measured the second register after (this has no effect on the outcome) but will get rid of the sum in 3 and therefore simplifies the notation

Comment 2: If $ \mathbf{s} = 0$ we have $\{\mathbf{s}\}^\perp = Z^n_2$ and stopping after obtaining $n-1$ linear independent vectors will not give the right result, but repeated application of the alogorithm will yield random results, which will show  $ \mathbf{s} = 0$ 

We are now in the position the implement the algorithm outlined above. We start by implementing the function. Here for simplicity, we chose f to be the identity, modified in such a way, that $f(\mathbf{x}) = f(\mathbf{y}) \iff (\mathbf{x} = \mathbf{y}\, \mathrm{or} \, \mathbf{x}= \mathbf{y} \oplus \mathbf{s})$

In [304]:
def f(q,s,n):
    for i in range(n):
        # f = id
        yield cirq.CNOT(q[i],q[i+n])
        #enforece condition on f by setting f(s) = 0 
        try:
            j = s.index(1)
        except:
            pass
        if s[i]:
             yield cirq.CNOT(q[j],q[i+n])

In [172]:
def simon_circuit(s):
    n = len(s)
    c = cirq.Circuit()
    q_l = cirq.LineQubit.range(2*n)
    # create |\psi_1>
    for i in range(n):
        c.append(cirq.H(q_l[i]))
    c.append(f(q_l,s,n))
    for i in range(n):
        c.append(cirq.H(q_l[i]))
    c.append(cirq.measure(*q_l[0:n], key='first_reg'))
    return c

In [335]:
print(simon_circuit([0,1,1,0,1]))

          ┌─────┐   ┌──┐
0: ───H────@─────────H─────────────────M('first_reg')───
           │                           │
1: ───H────┼@────────@─────@───@───H───M────────────────
           ││        │     │   │       │
2: ───H────┼┼@───────┼H────┼───┼───────M────────────────
           │││       │     │   │       │
3: ───H────┼┼┼@──────┼H────┼───┼───────M────────────────
           ││││      │     │   │       │
4: ───H────┼┼┼┼@─────┼H────┼───┼───────M────────────────
           │││││     │     │   │
5: ────────X┼┼┼┼─────┼─────┼───┼────────────────────────
            ││││     │     │   │
6: ─────────X┼┼┼─────X─────┼───┼────────────────────────
             │││           │   │
7: ──────────X┼┼───────────X───┼────────────────────────
              ││               │
8: ───────────X┼───────────────┼────────────────────────
               │               │
9: ────────────X───────────────X────────────────────────
          └─────┘   └──┘


We will use the SVD to check the rank of the matrix.
In principle, one could solve the linear system using Gaussian elimination over $Z_2$. Still, here it is easier to use an out of the box method working in $\mathbb{R}^n$ since we know the vectors will be parallel and we can just convert the non-zero entries to $1$.

In [351]:
def get_s(s):
    simulator =cirq.Simulator()
    n = len(s)
    c = simon_circuit(s)
    w_l = []
    for i in range(n-1):
        w_l.append(np.array(simulator.run(c).measurements['first_reg'][0]))
    sig = sp.linalg.svdvals(w_l)
    while np.count_nonzero(sig[sig>1e-3]) < n-1:
        w_l.append(np.array(simulator.run(c).measurements['first_reg'][0]))
        sig = sp.linalg.svdvals(w_l)
    sol = sp.linalg.null_space(w_l).T[0]
    sol = [1 if abs(i) >1e-3 else 0 for i in sol]
    return sol

In [352]:
get_s([0,1])

[0, 1]

In [353]:
get_s([0,1,1])

[0, 1, 1]

Note for longer $s$; we do not always get the correct answer immediately; here we have to re-run the algorithm several times and record the most common outcome. Strictly speaking, we have to do this anyway to account for the possibility $s=0$.  